<a href="https://colab.research.google.com/github/shengqin/20181008_Bioinformatics-Tools/blob/master/09_2022_cfRNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import string

In [ ]:
count = pd.read_csv(filepath_or_buffer='/content/drive/Othercomputers/My MacBook Pro/07-2022_SQ_HR/Stanford/Research/CIRCE_Project/data/cfrnamatrix_allgenes_ensemble_rawcount.txt', sep = "\t")
count = count[count.columns.drop(list(count.filter(regex='.10d|.fd|.30')))]
count.columns = [pname.replace(".10","_cfRNA").replace(".f","_PBMC") for pname in list(count)]

In [ ]:
# Keep rows with mean read counts >10
count_filtered = count[count.iloc[:, 1:].sum(axis=1)/(len(count.columns)-1) > 10]
count_filtered['gene'] = count_filtered['gene'].str.split('.').str[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
count_filtered.to_csv("/content/drive/Othercomputers/My MacBook Pro/07-2022_SQ_HR/Stanford/Research/CIRCE_Project/data/cfrnamatrix_rawcount_filtered.csv", index=False)

In [ ]:
count_filtered

,gene,CTR93_cfRNA,CTR93_PBMC,CTR94_cfRNA,CTR94_PBMC,CTR95_cfRNA,CTR95_PBMC,CTR96_cfRNA,CTR96_PBMC
2,ENSG00000000419,132,69,33,32,94,41,60,120
3,ENSG00000000457,118,62,29,21,75,48,70,120
4,ENSG00000000460,92,152,79,104,110,63,162,415
5,ENSG00000000938,1522,627,379,171,709,381,881,851
6,ENSG00000000971,40,14,5,0,22,5,8,5
...,...,...,...,...,...,...,...,...,...
19969,ENSG00000281028,77,107,33,30,62,45,63,127
19975,ENSG00000281887,428,112,89,12,202,96,146,173
19978,ENSG00000282034,242,90,8,21,154,105,0,0
19981,ENSG00000282278,220,404,109,149,274,257,668,2246


In [ ]:
# Get our lab gene list
filelocate = "/content/drive/Othercomputers/My MacBook Pro/07-2022_SQ_HR/Stanford/Research/CIRCE_Project"
count_DESeq_raw = pd.read_csv(filepath_or_buffer= filelocate + '/analysis/09-2022_cfRNAvsPBMC.csv', sep = ",")
count_DESeq_raw.rename(columns={count_DESeq_raw.columns[0]: "gene" }, inplace = True)
count_DESeq = count_DESeq_raw[(count_DESeq_raw['padj']<= 0.05)]
count_DESeq = count_DESeq[(count_DESeq['log2FoldChange']>= 1) | (count_DESeq['log2FoldChange']<= -1 )  ]
lab_DEGene = count_DESeq['gene']

In [ ]:
Up_cfRNA = count_DESeq[(count_DESeq['log2FoldChange']>= 1)]['gene']
Up_PBMC = count_DESeq[(count_DESeq['log2FoldChange']<= -1)]['gene']
Non_DEG = count_DESeq_raw[~count_DESeq_raw['gene'].isin(lab_DEGene)]['gene']

In [ ]:
print(len(Up_cfRNA),len(Up_PBMC),len(Non_DEG))

1008 569 10143


In [ ]:
# Get GTEx gene list
DEG_Artery = pd.read_csv(filelocate + '/data/DEG_ArteryvsNon_artery_paired.txt', sep = '\t')
DEG_Heart = pd.read_csv(filelocate + '/data/DEG_HeartvsNon_heart_paired.txt', sep = '\t')
DEG_Artery_fted = DEG_Artery[(DEG_Artery['logFC']>= 1) | (DEG_Artery['logFC']<= -1) & (DEG_Artery['adj.P.Val']<= 0.05)]
DEG_Heart_fted = DEG_Heart[(DEG_Heart['logFC']>= 1) | (DEG_Heart['logFC']<= -1) & (DEG_Heart['adj.P.Val']<= 0.05)]
GTEx_Heart_DEGene = DEG_Heart_fted['Gene']
GTEx_Artery_DEGene = DEG_Artery_fted['Gene']

In [ ]:
Heart_common = set(lab_DEGene).intersection(set(GTEx_Heart_DEGene))
Artery_common = set(lab_DEGene).intersection(set(GTEx_Artery_DEGene))
print(len(Heart_common),len(Artery_common))

350 363


In [ ]:
SpecificGene = pd.read_csv(filelocate+"/data/09-2022_PANDA_GeneList.csv", sep = ',')
SpecificGene = SpecificGene[(SpecificGene['Multiplicity']>= 1)].iloc[: , :5]
SpecificGeneList = SpecificGene.iloc[: , :5]['Name']

In [ ]:
print(len(set(Up_cfRNA)&set(SpecificGeneList)),len(set(Up_PBMC)&set(SpecificGeneList)), len(set(Non_DEG)&set(SpecificGeneList)))

286 175 3009


In [ ]:
len(SpecificGeneList)

12586

In [ ]:
Ajay_gene = pd.read_csv(filelocate + "/data/Artery_Coronary_qQTL_gene.txt", sep = "\t")
Ajay_list = Ajay_gene['EQTLgene'].str.split('.').str[0]
Ajay_DESeq = set(lab_DEGene)&set(Ajay_list)

In [ ]:
count_DESeq_raw[count_DESeq_raw['gene'].isin(list(Ajay_list))]

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
4400,ENSG00000129354,536.088614,-0.406129,0.229432,-1.770150,0.076702,0.168091
5868,ENSG00000143126,27.481646,-0.914663,0.416720,-2.194912,0.028170,0.079940
8016,ENSG00000166197,153.347923,0.620075,0.246630,2.514188,0.011931,0.041162
8031,ENSG00000166275,26.045602,0.989935,0.404935,2.444675,0.014498,0.047932
11122,ENSG00000214435,10.433659,1.167361,0.687088,1.698999,0.089319,0.187689


In [ ]:
Uniq_Tissue = list(set([j for sub in SpecificGene['Tissues'].str.split(',') for j in sub]))
Uniq_Tissue_list = ['Adipose', 'Adrenal_gland', 'Artery', 'Brain', 'Breast', 'Colon', 'Esophagus', 'Fibroblast_cell_line', 
                    'Gastroesophageal_junction', 'Heart', 'Intestine_terminal_ileum', 'Kidney_cortex', 'Liver', 'Lung', 'Lymphoblastoid_cell_line', 
                    'Minor_salivary_gland', 'Ovary', 'Pancreas', 'Pituitary', 'Prostate', 'Skeletal_muscle', 'Skin', 'Spleen', 'Stomach', 'Testis', 
                    'Thyroid', 'Tibial_nerve', 'Uterus', 'Vagina', 'Whole_blood']


In [ ]:
# Get cfRNA enrichment for all tissues
T_Specific = pd.DataFrame()
T_Specific['Tissue'] = ['Up_cfRNA','Up_PBMC','Non_DEGs']
whole_DEG = [Up_cfRNA, Up_PBMC, Non_DEG]
temp = [0]*3
for tissue in Uniq_Tissue_list:
  tissue_list = SpecificGene[SpecificGene['Tissues'].str.contains(tissue)]['Name']
  for j in range(len(T_Specific['Tissue'])):
    temp[j] = round(len(set(whole_DEG[j])&set(tissue_list)))
    #/len(whole_DEG[j]), 4)
  T_Specific[tissue] = temp



In [ ]:
# Prepare TS_Percent dataframe
TS_Percent = pd.DataFrame()
# Drop tissues with sum <= 7
for column in T_Specific.iloc[:, 1:]:
    if (T_Specific[column].sum() <= 7):
        T_Specific.drop(column, inplace=True, axis=1)

for index, row in T_Specific.iterrows():
  TS_Percent.at[index, 'Tissue'] = row['Tissue']+"%"

TS_Percent = pd.concat([TS_Percent, round(T_Specific.iloc[:, 1:]/T_Specific.iloc[:, 1:].sum(axis = 0),4)], axis=1)

In [ ]:
TS_Percent_ggplot = pd.melt(TS_Percent, id_vars='Tissue', value_vars=list(TS_Percent.columns)[1:], value_name='Percent')
TS_Percent_ggplot.to_csv(filelocate + '/data/TS_Percent_ggplot.csv')

['Adrenal_gland',
 'Artery',
 'Brain',
 'Colon',
 'Esophagus',
 'Fibroblast_cell_line',
 'Heart',
 'Intestine_terminal_ileum',
 'Kidney_cortex',
 'Liver',
 'Lung',
 'Lymphoblastoid_cell_line',
 'Minor_salivary_gland',
 'Ovary',
 'Pancreas',
 'Pituitary',
 'Prostate',
 'Skeletal_muscle',
 'Skin',
 'Spleen',
 'Stomach',
 'Testis',
 'Thyroid',
 'Tibial_nerve',
 'Vagina',
 'Whole_blood']

In [ ]:
print(sorted(Uniq_Tissue))

['', 'Adipose_subcutaneous', 'Adipose_visceral', 'Adrenal_gland', 'Artery_aorta', 'Artery_coronary', 'Artery_tibial', 'Brain_basal_ganglia', 'Brain_cerebellum', 'Brain_other', 'Breast', 'Colon_sigmoid', 'Colon_transverse', 'Esophagus_mucosa', 'Esophagus_muscularis', 'Fibroblast_cell_line', 'Gastroesophageal_junction', 'Heart_atrial_appendage', 'Heart_left_ventricle', 'Intestine_terminal_ileum', 'Kidney_cortex', 'Liver', 'Lung', 'Lymphoblastoid_cell_line', 'Minor_salivary_gland', 'Ovary', 'Pancreas', 'Pituitary', 'Prostate', 'Skeletal_muscle', 'Skin', 'Spleen', 'Stomach', 'Testis', 'Thyroid', 'Tibial_nerve', 'Uterus', 'Vagina', 'Whole_blood']


In [ ]:
Artery_specificgene = SpecificGene[SpecificGene['Tissues'].str.contains('Artery')]['Name']
Heart_specificgene = SpecificGene[SpecificGene['Tissues'].str.contains('Heart')]['Name']
Lung_specificgene = SpecificGene[SpecificGene['Tissues'].str.contains('Lung')]['Name']
CV_specificgene = (list(Artery_specificgene)+list(Heart_specificgene))

In [ ]:
len(CV_specificgene)

610

In [ ]:
# Heart specificity
print(len(set(Up_cfRNA)&set(CV_specificgene)),len(set(Up_PBMC)&set(CV_specificgene)), len(set(Non_DEG)&set(CV_specificgene)))

23 7 123


In [ ]:
# Lung specificity
print(len(set(Up_cfRNA)&set(Lung_specificgene)),len(set(Up_PBMC)&set(Lung_specificgene)), len(set(Non_DEG)&set(Lung_specificgene)))

3 3 19


## Why is cfRNA important
* 390/409 = up & down, next is to break them up/down seperately.
* Will have patient samples, compare them.
* From GTEx, get tissue specific genes for other tissues.

## Get the following number
* percent of tissue specific gene = (multiplicity>=1 & Up_DEGs)/Up_DEGs
* percent of PBMC specific gene = (multiplicity>=1 & Down_DEGs)/Down_DEGs
* percent of non specific gene = (multiplicity>=1 & Down_DEGs)/Down_DEGs

In [ ]:
count_DESeq_raw

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,ENSG00000000419,64.204073,0.368349,0.300316,1.226540,0.219995,0.360759
1,ENSG00000000457,58.886319,0.371608,0.303147,1.225834,0.220261,0.360993
2,ENSG00000000460,129.630404,-0.362594,0.262511,-1.381253,0.167201,0.296819
3,ENSG00000000938,608.829748,0.907757,0.210174,4.319081,0.000016,0.000185
4,ENSG00000000971,10.481578,1.774244,0.668779,2.652960,0.007979,0.030302
...,...,...,...,...,...,...,...
11715,ENSG00000281028,60.476555,-0.178027,0.304206,-0.585217,0.558402,0.689942
11716,ENSG00000281887,136.345008,1.373928,0.329997,4.163462,0.000031,0.000331
11717,ENSG00000282034,71.603687,0.177656,0.415157,0.427924,0.668707,0.777659
11718,ENSG00000282278,391.261123,-0.690109,0.214803,-3.212750,0.001315,0.007372


In [ ]:
count_DESeq

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
4,ENSG00000000971,10.481578,1.774244,0.668779,2.652960,7.978937e-03,3.030238e-02
10,ENSG00000001497,82.118334,1.138183,0.300463,3.788093,1.518081e-04,1.253834e-03
27,ENSG00000003400,49.684587,1.152768,0.330067,3.492523,4.784812e-04,3.268173e-03
38,ENSG00000004478,77.675944,1.176359,0.302893,3.883749,1.028582e-04,9.146421e-04
45,ENSG00000004779,49.921107,1.702384,0.333377,5.106488,3.282012e-07,6.701251e-06
...,...,...,...,...,...,...,...
11710,ENSG00000280071,141.089535,2.095737,0.279089,7.509208,5.948604e-14,5.164270e-12
11711,ENSG00000280571,78.406114,-6.139415,1.463023,-4.196389,2.712043e-05,2.918746e-04
11712,ENSG00000280789,16.464384,1.340035,1.581134,0.847515,3.967082e-01,5.460912e-01
11714,ENSG00000280987,729.351954,-1.264114,0.281121,-4.496694,6.901823e-06,9.274084e-05
